<a href="https://colab.research.google.com/github/williamlidberg/Unet-tutorial/blob/main/U_net_tutorial_with_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection of Hunting Pits usng Airborne Laser Scanning and Deep Learning
This is an example from the [paper](https://www.tandfonline.com/doi/full/10.1080/00934690.2024.2364428) adapted to google colab. [Data](https://snd.se/en/catalogue/dataset/2023-188) and [code](https://github.com/williamlidberg/Detection-of-hunting-pits-using-airborne-laser-scanning-and-deep-learning) are openly avalible online.

# Intro to google colab

# Cells
A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

## Code cells
Below is a **code cell**. Once the toolbar button indicates CONNECTED, click in the cell to select it and execute the contents in the following ways:

* Click the **Play icon** in the left gutter of the cell;
* Type **Cmd/Ctrl+Enter** to run the cell in place;
* Type **Shift+Enter** to run the cell and move focus to the next cell (adding one if none exists); or
* Type **Alt+Enter** to run the cell and insert a new code cell immediately below it.

There are additional options for running some or all cells in the **Runtime** menu.


In [ ]:
a = 10
a

# Working with python
Colaboratory is built on top of [Jupyter Notebook](https://jupyter.org/). Below are some examples of convenience functions provided.

In [ ]:
import time
print("Sleeping")
time.sleep(10) # sleep for a while; interrupt me!
print("Done Sleeping")

## Adding and moving cells
You can add new cells by using the **+ CODE** and **+ TEXT** buttons that show when you hover between cells. These buttons are also in the toolbar above the notebook where they can be used to add a cell below the currently selected cell.

You can move a cell by selecting it and clicking **Cell Up** or **Cell Down** in the top toolbar.

Consecutive cells can be selected by "lasso selection" by dragging from outside one cell and through the group.  Non-adjacent cells can be selected concurrently by clicking one and then holding down Ctrl while clicking another.  Similarly, using Shift instead of Ctrl will select all intermediate cells.

# Build your first neural network
We will download some classified data from the internet to use as training data for your first model.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Plot 25 sample images from the training set
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([]), plt.yticks([]), plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(class_names[y_train[i][0]])
plt.tight_layout()
plt.show()

### Build a neural network

In [ ]:
from tensorflow.keras import layers, models

# Define the shape of the input images (flattened 32x32x3)
image_shape = (32 * 32 * 3,)

# Build the model
model = models.Sequential([
    layers.Input(shape=image_shape),                # Input layer
    layers.Dense(32, activation='relu'),           # Hidden layer with 32 neurons
    layers.Dense(10)                                # Output layer 10 classes: 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

### Train the model

In [ ]:
# Flatten the images and normalize pixel values
x_train_flat = x_train.reshape(len(x_train), -1) / 255.0
x_test_flat = x_test.reshape(len(x_test), -1) / 255.0

# Train the model and save the history
history = model.fit(x_train_flat, y_train, epochs=10,
                    validation_data=(x_test_flat, y_test))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test_flat, y_test, verbose=2)

We can look at the loss curves to see if the model overfits

In [ ]:
# Plot training & validation loss and accuracy
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Deep learning
Now we will increase the models capacity to learn by adding more hidden layers with neurons. This is called deep learning.

In [ ]:
image_shape = (32 * 32 * 3,)

# Build the model
model = models.Sequential([
    layers.Input(shape=image_shape),                # Input layer
    layers.Dense(128, activation='relu'),           # Hidden layer with 128 neurons
    layers.Dense(64, activation='relu'),            # Hidden layer with 64 neurons
    layers.Dense(32, activation='relu'),            # Hidden layer with 32 neurons
    layers.Dense(10)                                # Output layer 10 classes: 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model and save the history
history = model.fit(x_train_flat, y_train, epochs=10,
                    validation_data=(x_test_flat, y_test))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test_flat, y_test, verbose=2)

Did the model improve?

# Convolutional Neural networks
So far we have used a one dimensional neural network that took flatted images as input. The next model will introduce convolutional layers. A convolutional layer is a neural network layer that automatically learns spatial patterns by applying filters to local regions of input data, such as images. Convolutional layers work by sliding learnable filters (kernels) over the input data to detect patterns like edges or textures, producing feature maps that highlight important spatial features. \
\
\
This is an example of a kernel sliding over an image.


![Alt Text](https://maucher.home.hdm-stuttgart.de/Pics/gif/no_padding_no_strides.gif)



We will start by replacing the flattend input from our previous model with the 2 d images with 3 channels. Next we will add a convolutional layer with the size 3 by 3 pixels. This layer will extract 32 feature layers that will be pooled and used as input for the fully connected neural network. We will also introduce max pooling and dropout. Max pooling is a way to select the maximum value for each region in the convolutional filter and dropout helps with overfitting.  

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# we will not flatten the images this time.
x_train = x_train / 255.0
x_test = x_test / 255.0

input_shape=(32, 32, 3) # the input shape is no longer flat.

# Build the model
model = models.Sequential([
    layers.Input(input_shape),                     # Input for CIFAR-10 shape
    layers.Conv2D(32, (3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D((2, 2)),                   # max pooling
    layers.Dropout(0.5),
    layers.Flatten(),                              # flatten 2d data to 1 d
    layers.Dense(128, activation='relu'),          # hidden layer with 128 neurons
    layers.Dense(10)                               # Output layer
])
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model and save the history
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

In [ ]:
# Plot training & validation loss and accuracy
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Deep learning with cultural remains
In this tutorial you will learn:


1.   Google colab
2.   Import python libraries
3.   Run python code
6.   Build a Unet model
7.   Train and test a Unet model
8.   Transfer learning


## Import data
These two cells downloads a dataset from williams google drive. The first cell is long and complicated due to googles shenanigans. The second cell unpacks the data into google colabs file tree.

In [ ]:
import gdown
file_id = "1EQKhlQFjg0mP7OfoT0NRURQZAgRAqBvj"
output = "earth.zip"
gdown.download(
    f"https://drive.google.com/uc?export=download&confirm=pbef&id={file_id}",
    output
)

In [ ]:
!unzip /content/earth.zip -d /content/

In [ ]:
import os # For data handling
import glob # For data handling
import numpy as np # For arrays
import keras # for making models
import visualkeras # for showing models
import tensorflow.keras.backend as K # For deep learning
import tensorflow as tf # For deep learning
import matplotlib.pyplot as plt # For plotting data
import random # for seeds
random.seed(10) # To increase reproducability

## Inspect data
The first stepp in any data analysis is always to inspect the data. The data in this tutorial is stored as a tensorflow dataset. If you want to get into the details on how to create a training dataset you can check out the github linked to the research paper: https://github.com/williamlidberg/Detection-of-hunting-pits-using-airborne-laser-scanning-and-deep-learning

In [ ]:
train_images = tf.data.Dataset.load('/content/earth/train/') # These will be used to train the model
test_images = tf.data.Dataset.load('/content/earth/test/') # These will be used to test the model to get a estimate of the preformance.
train_examples = list(train_images.as_numpy_iterator())
print(len(train_images), 'images will be used to train the model.')
print(len(test_images), 'images will be used to test the model.')

Plot some examples from the training data to make sure it looks resonable. Hunting pits a tiny feaures and on the limit of what can be seen in this digital elevation model.

In [ ]:
example1 = np.squeeze(train_examples[0]) # image number 0
image1 = example1[0,:,:,]
label1 = example1[1,:,:,]

example2 = np.squeeze(train_examples[42]) # image number 42
image2 = example2[0,:,:,]
label2 = example2[1,:,:,]

example3 = np.squeeze(train_examples[1000])  # image number 1000
image3 = example3[0,:,:,]
label3 = example3[1,:,:,]

# select and plot images
plt.rcParams['figure.figsize'] = [12, 12]
f, axarr = plt.subplots(3,2)
axarr[0,0].set_title('Hillshade')
axarr[0,0].imshow(image1, cmap='Greys_r')
axarr[0,1].set_title('Labeled hunting pit')
axarr[0,1].imshow(label1,cmap='Greys')

axarr[1,0].set_title('Hillshade')
axarr[1,0].imshow(image2, cmap='Greys_r')
axarr[1,1].set_title('Labeled hunting pit')
axarr[1,1].imshow(label2,cmap='Greys')

axarr[2,0].set_title('Hillshade')
axarr[2,0].imshow(image3, cmap='Greys_r')
axarr[2,1].set_title('Labeled hunting pit')
axarr[2,1].imshow(label3,cmap='Greys')

### Are points real hunting pits?

## Build input pipelines
[Batch size](https://medium.com/deep-learning-experiments/effect-of-batch-size-on-neural-net-training-c5ae8516e57) is a term used in machine learning and refers to the number of training examples utilized in one iteration. Larger batch sizes is genreally better but GPU memory usually very limited.

Buffer size is how many images that should be read into system memory (RAM) at the time. For a large number of images it is not possible to read the entire dataset into the system memory but having a small buffer ready speeds up the training process. Otherwise the images needs to be read from disk which is much slower.

**Training pipeline**

In [ ]:
BATCH_SIZE = 16
BUFFER_SIZE = 32
train_batches = (train_images
                    .cache() # cache data
                    .shuffle(BUFFER_SIZE) # fill buffer, sample from it and replace with new items (buffer size > training set for perfect shuffling)
                    .batch(BATCH_SIZE)  # set batch size
                    .repeat()  # repeat dataset idefinetely
                    .prefetch(buffer_size=32)) # prefetch 100 images to optimize runtime

**Testing pipeline**

In [ ]:
# setup input pipeline
BATCH_SIZE = 16
BUFFER_SIZE = 32
test_batches = (test_images
                    .cache() # cache data
                    .shuffle(BUFFER_SIZE) # fill buffer, sample from it and replace with new items (buffer size > training set for perfect shuffling)
                    .batch(BATCH_SIZE)  # set batch size
                    .repeat(1)  # repeat dataset idefinetely
                    .prefetch(buffer_size=32)) # prefetch 100 images to optimize runtime

## Define metrics to use for evaluation
Accuracy is how many of the pixels are classified correctly by the model. This is not very usefull if most of the landscape consists of one class. In this case most of the forest are not hunting pits.

f1 score is a better option since it tries to take the inbalance of the data into account. f1 = 0 is very low and f1 = 1 is a perfect fit. Both recall and precision are required to calculate F1 score so these have to be defined as well.

In [ ]:
import tensorflow.keras.backend as K

def recall_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')  # Ensure both are float32
    y_pred = K.cast(y_pred, 'float32')  # Ensure both are float32
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')  # Ensure both are float32
    y_pred = K.cast(y_pred, 'float32')  # Ensure both are float32
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


## Build U-net model
[U-net](https://arxiv.org/abs/1505.04597) is a convolutional neural network that was developed for biomedical image segmentation. The network consists of a contracting path and an expansive path, which gives it the [u-shaped architecture](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/). The contracting path is a typical convolutional network that consists of repeated application of convolutions, each followed by a rectified linear unit (ReLU) and a max pooling operation. During the contraction, the spatial information is reduced while feature information is increased. The expansive pathway combines the feature and spatial information through a sequence of up-convolutions and concatenations with high-resolution features from the contracting path.

In [ ]:
#Build the model
IMG_WIDTH = 256 # image width in number of pixels
IMG_HEIGHT = 256 # image height in number of pixels
IMG_CHANNELS = 1 # a grey scale image only has one band. Normal images (RGB) have three channels.
NUM_CLASSES = 1 # The images are labaled into 0 and 1 where 0 = no hunting pit and 1 = hunting pit

inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

#Contraction path
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs) # 32 is the number of feature maps and 3, 3 is the size of the convulutional filter.
c1 = tf.keras.layers.Dropout(0.1)(c1) # dropouts combats overfitting
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(NUM_CLASSES, (1, 1))(c9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])


We can also print out a text summary of the model. Note that the first and last layers are both 256x256 just like our resized data. The 1 in the first layer represents the number of channels. A RGB image would have 3 channels but since this is a greyscale it only has 1.

That might be a bit dense to understand as a beginer. Another alternative is to make a figure of the model instead.

In [ ]:
visualkeras.layered_view(model, legend=True)

## Train the U-net
Now we are ready to train the UNet. Epoch is one pass of the model across the training data. Training the model for more epochs takes longer time but gives more accurate results.

The “steps per epoch” parameter is a value that defines the total number of steps taken before one epoch is finished and the next epoch starts during training. It is typically set to the total number of samples in your dataset divided by the batch size. 10 epochs takes around 5min to train on Colabs standard GPU.

In [ ]:
steps = len(train_images)//BATCH_SIZE # len means number of images in the training data. Batch size was defined above.
original_model = model
original_model.compile(tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=True), metrics=['acc', f1_m, recall_m])
result = original_model.fit(train_batches, epochs=5, steps_per_epoch=steps)

## Plot the training history

In [ ]:
# summarize history for accuracy
plt.rcParams['figure.figsize'] = [8, 8]
plt.plot(result.history['f1_m'])
plt.title('Model accuracy using f1 score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.xlim(0,10)
plt.ylim(0,1)
plt.show()


## Evaluate the model by applying it to the test images

In [ ]:
results = original_model.evaluate(test_batches)

The model seems to strugle to learn how to detect hunting pits. Probably due to a combination of a low number of training images and the fact that hunting pits are very small.

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
f, axarr = plt.subplots(1,3)

test_examples = list(test_images.as_numpy_iterator())
test_example1 = np.squeeze(test_examples[1])
original = test_example1[0,:,:,]
label = test_example1[1,:,:,]

testimage = np.reshape(original,[1,256,256,1])
prediction = original_model.predict(testimage)
pred = np.squeeze(prediction)

# Image
axarr[0].set_title('Original image')
axarr[0].imshow(original, cmap='Greys_r')

# Label
axarr[1].set_title('Labeled pits')
axarr[1].imshow(label, cmap='Greys_r')

# Prediction
axarr[2].set_title('Predicted pits')
axarr[2].imshow(pred)

We can set pred>0.5 to get a binary prediction just like the label

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
f, axarr = plt.subplots(1,3)

test_examples = list(test_images.as_numpy_iterator())
test_example1 = np.squeeze(test_examples[1])
original = test_example1[0,:,:,]
label = test_example1[1,:,:,]

testimage = np.reshape(original,[1,256,256,1])
prediction = original_model.predict(testimage)
pred = np.squeeze(prediction)

# Image
axarr[0].set_title('Original image')
axarr[0].imshow(original, cmap='Greys_r')

# Label
axarr[1].set_title('Labeled pits')
axarr[1].imshow(label, cmap='Greys_r')

# Prediction
axarr[2].set_title('Predicted pits')
axarr[2].imshow(pred>0.5, cmap='Greys_r')

# Transfer learning

Transfer learning is a method where a model is first trained on different, but similair, data and then fine-tuned on the real data. This means that the model does not need to start from scratch when training on the real data.

In our case we need a dataset with something similair to hunting pits. Luckley for us there is a place with plenty of pits that we can use. The moon!


## Download the trained moon model
Start by cloning the github repository where the trained model is stored.

In [ ]:
!git clone https://github.com/williamlidberg/Detection-of-hunting-pits-using-airborne-laser-scanning-and-deep-learning.git /content/code/

Load the trained model and use it the moon

In [ ]:
model.load_weights('/content/code/models/moon_hillshade.h5', by_name=True, skip_mismatch=True)  # Load weights into 'model'
pre_trained_model = model
pre_trained_model.compile(tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=True), metrics=['acc', f1_m, recall_m])

# Train the lunar model on data of hunting pits

In [ ]:
steps = len(train_images) // BATCH_SIZE
result_pretrained = pre_trained_model.fit(train_batches, validation_data=test_batches, epochs=10, steps_per_epoch=steps)  # Use 'model' for training

In [ ]:
# summarize history for accuracy
plt.rcParams['figure.figsize'] = [8, 8]
plt.plot(result_pretrained.history['f1_m'])
plt.title('Model accuracy using f1 score')
plt.ylabel('f1 score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.xlim(0,10)
plt.ylim(0,1)
plt.show()

You could train the model for longer and get even better results but at some point the curve will level out. Test the model on the test data and compare the results to the original model.

In [ ]:
pre_trained_model.evaluate(test_batches)

Now we can take the new model and try it on a test image.

In [ ]:
plt.rcParams['figure.figsize'] = [16, 16]
f, axarr = plt.subplots(1,4)

test_examples = list(test_images.as_numpy_iterator())
test_example1 = np.squeeze(test_examples[1])
original = test_example1[0,:,:,]
label = test_example1[1,:,:,]

testimage = np.reshape(original,[1,256,256,1])
# original model
prediction = original_model.predict(testimage)
pred = np.squeeze(prediction)

#pre trained model
prediction_pretrained = pre_trained_model.predict(testimage)
pred_pretrained = np.squeeze(prediction_pretrained)

# Image
axarr[0].set_title('Original image')
axarr[0].imshow(original, cmap='Greys_r')

# Label
axarr[1].set_title('Labeled pits')
axarr[1].imshow(label, cmap='Greys_r')

# Prediction from original model
axarr[2].set_title('Predicted pits original model')
axarr[2].imshow(pred)#>0.5, cmap='Greys_r')

# Prediction
axarr[3].set_title('Predicted pits pre-trained model')
axarr[3].imshow(pred_pretrained)#>0.5, cmap='Greys_r')

## Questions



1.   Will including more images produce better results?
2.   Will an increased batch size improve the model?
3.   Will the model preform better when training for more epochs?
4.   Does training the original model for twice as lone result in the same accuracy as the pre-trained model?





---



---


## Reading

https://arxiv.org/abs/1505.04597

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d

https://naokishibuya.medium.com/up-sampling-with-transposed-convolution-9ae4f2df52d0


## Contact
William.lidberg@slu.se




